# Cineworld Listings

I like Weird movies coming to Imax, I don't like watching newspapers/etc to see Belfast announcing them. 

## Basic mode

Yup, turns out cineworld just checks based on user-agent. And overall appears to be even more basic that [what this one looked like](https://github.com/oracal/cineworld)

Easy observations:

* Films are uniquely identified by `id`, which is also persisted in teh rest of the api calls (see `link` attribute)
* `117` is Belfasts Site code
* No clue what the 10108 is.

In [ ]:
import requests
from datetime import date

headers = {
    "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:124.0) Gecko/20100101 Firefox/124.0",
}

requests.get(f"https://www.cineworld.co.uk/uk/data-api-service/v1/quickbook/10108/film-events/in-cinema/117/at-date/{date.today().isoformat()}", 
             headers=headers).json()['body']['films'][0]

In [ ]:
from datetime import date
def get_cinema_listing_for(screening_date = None, site_code=117):
    if screening_date is None:
        screening_date = str(date.today())
    elif isinstance(screening_date, date):
        screening_date = str(screening_date)
    else:
        raise ValueError("Can only use date-type with screening_date")
    response = requests.get(f"https://www.cineworld.co.uk/uk/data-api-service/v1/quickbook/10108/film-events/in-cinema/{site_code}/at-date/{screening_date}", headers=headers)
    response.raise_for_status()
    return response.json()['body']['films']
    
listings = get_cinema_listing_for()
listings[0]['name']

In [ ]:
listings[0].keys()

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

listings = []

for d in tqdm(pd.date_range('today', freq='D', periods=28)):
    _d = d.date()
    for listing in get_cinema_listing_for(_d):
        listing['date'] = _d
        listings.append(listing)        

In [ ]:
df = pd.DataFrame(listings)

In [ ]:
df[df['attributeIds'].apply(lambda l: 'imax' in l)][['name','date']]